<a href="https://colab.research.google.com/github/akutayaydin/Magnimind-6-Image-Processing/blob/main/TNT_Project_v8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import cv2
import os
from PIL import Image
from skimage import io, filters
import scipy.ndimage as ndi
import sys
import keras.utils as image
from tensorflow.keras.utils import load_img
from google.colab.patches import cv2_imshow

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
unlabeled_image = cv2.imread('/content/gdrive/My Drive/Google Colab Folder/TNT_Project/m05.png')
#unlabeled_image.dtype

In [ ]:
#Apply Filters

In [ ]:
#Rolling Ball Algorithm : to remove uneven backgrounds or smooth the intensity variations in an image
# where the image may have uneven lighting or background artifacts

In [ ]:
import cv2
import numpy as np

def background_and_shading_correction(image, ball_radius):
    # Convert the image to grayscale, rolling ball algorithm is usually applied to grayscale images.
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply the rolling ball algorithm for background correction
    #The kernel_size is set to be twice the ball_radius plus one.
    #The "+1" ensures that the kernel has an odd size, which is a requirement for morphological operations.
    kernel_size = ball_radius * 2 + 1
    background_corrected = cv2.morphologyEx(gray_image, cv2.MORPH_TOPHAT, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (kernel_size, kernel_size)))

    # Perform shading correction by subtracting the background corrected image from the original image
    shading_corrected = cv2.subtract(gray_image, background_corrected)

    # Convert the shading corrected image back to color (optional)
    shading_corrected_color = cv2.cvtColor(shading_corrected, cv2.COLOR_GRAY2BGR)

    # Combine the shading-corrected image with the original image to get the final result
    result_image = cv2.addWeighted(image, 1, shading_corrected_color, 0.7, 0)

    return result_image

# Load your input image
input_image = cv2.imread("/content/gdrive/My Drive/Google Colab Folder/TNT_Project/m05.png")

# Set the radius for the rolling ball algorithm (adjust as needed)
ball_radius = 50

# Perform background and shading correction
output_image = background_and_shading_correction(input_image, ball_radius)

# Save the processed image (replace "output_image.jpg" with the desired output path)
cv2.imwrite("/content/gdrive/My Drive/Google Colab Folder/TNT_Project/output_image_rolling_ball_50.png", output_image)



True

In [ ]:
import cv2

def background_and_shading_correction(image, sigma):
    # Convert the image to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply the Gaussian filter for background correction
    background_corrected = cv2.GaussianBlur(gray_image, (0, 0), sigmaX=sigma, sigmaY=sigma)

    # Perform shading correction by subtracting the background corrected image from the original image
    shading_corrected = cv2.subtract(gray_image, background_corrected)

    # Convert the shading corrected image back to color (optional)
    shading_corrected_color = cv2.cvtColor(shading_corrected, cv2.COLOR_GRAY2BGR)

    # Combine the shading-corrected image with the original image to get the final result
    result_image = cv2.addWeighted(image, 1, shading_corrected_color, 0.7, 0)

    return result_image


# Load input image
input_image = cv2.imread("/content/gdrive/My Drive/Google Colab Folder/TNT_Project/m05.png")

# Set the sigma value for the Gaussian filter (adjust as needed)
sigma = 3

# Perform background and shading correction
output_image = background_and_shading_correction(input_image, sigma)

# Save the processed image (replace "output_image.jpg" with the desired output path)
cv2.imwrite("/content/gdrive/My Drive/Google Colab Folder/TNT_Project/output_image_gaussian.png", output_image)


True

In [24]:
#Step 1:Read and Slice the Preprocessed Image
#Step 1.1: Slice the unlabel image in the steps of region size

In [100]:
img_preprocessed = cv2.imread("/content/gdrive/My Drive/Google Colab Folder/TNT_Project/output_image_rolling_ball_50.png")

#Set the region size
region_size = 512

#Get the dimensions of the image
height, width, channels = img_preprocessed.shape

#Iterate through the image in the steps of region_size
for y in range(0,height,region_size):
  for x in range (0,width,region_size):
    #Extract the region
    region = img_preprocessed[y:y+region_size,x:x+region_size]

    #Label the image based on the coordinates
    image_name = 'IMG' + '_'+ str(y) + '_' + str(x)

    #Create the image
    cv2.imwrite("/content/gdrive/My Drive/Google Colab Folder/TNT_Project/sliced_unlabeled_images/" + image_name + ".png", region)

In [ ]:
#Step 1.2: Slice the labeled image in the steps of region_size

In [101]:
img_labeled =  cv2.imread("/content/gdrive/My Drive/Google Colab Folder/TNT_Project/m05-label.png")

#Set the region size
region_size =512

for y in range(0,height,region_size):
  for x in range(0,width,region_size):
    #Extract the region
    region = img_labeled[y:y+region_size,x:x+region_size]

    #label the image based on the coordinates
    image_name = 'IMG' + '_' + str(y) + '_' + str(x)

    #Create and save the image file
    cv2.imwrite("/content/gdrive/My Drive/Google Colab Folder/TNT_Project/sliced_labeled_images/" + image_name + ".png", region)


In [47]:
#Step 2: Identify if there is tnt tube (yellow pixel) on the labeled image and update file name

In [103]:
folder_path = '/content/gdrive/My Drive/Google Colab Folder/TNT_Project/sliced_labeled_images/'
folder_path_unlabeled = '/content/gdrive/My Drive/Google Colab Folder/TNT_Project/sliced_unlabeled_images/'

lower_bound = np.array([0,100,100])  #lower bound BGR values for color
upper_bound = np.array([50,255,255]) #upper bound BGR values for color

#list all file names and create a mask for the target color
for f in os.listdir(folder_path):
  img_labeled_file = cv2.imread(folder_path +f)
  tnt_mask = cv2.inRange(img_labeled_file,lower_bound,upper_bound)

  # Count the number of color pixels
  color_pixel_count = cv2.countNonZero(tnt_mask)

  if color_pixel_count == 0 and '_tnt_0.png' not in f:
    #rename file name
    f_new = f.replace ('.png','_tnt_0.png')
    os.rename (folder_path + f,folder_path + f_new)
    os.rename (folder_path_unlabeled + f,folder_path_unlabeled + f_new)
    folder_path_unlabeled
  elif color_pixel_count > 0 and '_tnt_1.png' not in f:
    #rename file name
    f_new = f.replace ('.png','_tnt_1.png')
    os.rename (folder_path + f,folder_path + f_new)
    os.rename (folder_path_unlabeled + f,folder_path_unlabeled + f_new)

In [63]:
color_pixel_count

0